In [7]:
# References
#https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e
#https://github.com/MrFuguDataScience/JSON/blob/master/Nested_Json_Mult_Ex.ipynb
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.build_table_schema.html

import pandas as pd
import json

#### The following JSON contains information that is nested twice and is itself in a list - so one file can contain multiple JSON objects. 

Here is the structure:

In [50]:
with open("SampleRoutingData.json") as f:
    data =json.load(f)

[{'EventProperties': {'EnquedMessageTime': 1607243131479, 'MessageType': 'IoTHubRouting'}, 'Body': [{'msgtype': 'TELEMETRY DATA', 'Timestamp': 1607243134759, 'payload': [{'ID': 1, 'Sensor': 'Temp', 'Value': 30, 'Unit': 'Celsius'}, {'ID': 2, 'Sensor': 'Current', 'Value': 58, 'Unit': 'Volt'}]}, {'msgtype': 'TELEMETRY DATA', 'Timestamp': 1607243132333, 'payload': [{'ID': 3, 'Sensor': 'Temp', 'Value': 28.8, 'Unit': 'Celsius'}, {'ID': 4, 'Sensor': 'Current', 'Value': 43, 'Unit': 'Volt'}]}]}, {'EventProperties': {'EnquedMessageTime': 1607243131485, 'MessageType': 'IoTHubRouting'}, 'Body': [{'msgtype': 'TELEMETRY DATA', 'Timestamp': 1607243134759, 'payload': [{'ID': 5, 'Sensor': 'Temp', 'Value': 30, 'Unit': 'Celsius'}, {'ID': 6, 'Sensor': 'Current', 'Value': 58, 'Unit': 'Volt'}]}, {'msgtype': 'TELEMETRY DATA', 'Timestamp': 1607243132333, 'payload': [{'ID': 7, 'Sensor': 'Temp', 'Value': 28.8, 'Unit': 'Celsius'}, {'ID': 8, 'Sensor': 'Current', 'Value': 43, 'Unit': 'Volt'}]}]}]


In [43]:
# Read in JSON 
df = pd.json_normalize(data)

# Explode list of measurements in Body
df = df.explode('Body')
print(df)

                                                Body  \
0  {'msgtype': 'TELEMETRY DATA', 'Timestamp': 160...   
0  {'msgtype': 'TELEMETRY DATA', 'Timestamp': 160...   
1  {'msgtype': 'TELEMETRY DATA', 'Timestamp': 160...   
1  {'msgtype': 'TELEMETRY DATA', 'Timestamp': 160...   

   EventProperties.EnquedMessageTime EventProperties.MessageType  
0                      1607243131479               IoTHubRouting  
0                      1607243131479               IoTHubRouting  
1                      1607243131485               IoTHubRouting  
1                      1607243131485               IoTHubRouting  


In [44]:
df=pd.json_normalize(json.loads(df.to_json(orient="records")))
df=df.explode('Body.payload')

df = pd.concat([df.drop(['Body.payload'], axis=1), df['Body.payload'].apply(pd.Series)], axis=1)

In [49]:

df['Body.Timestamp'] = pd.to_datetime(df['Body.Timestamp'], unit='ms')